# Structured output

OpenAI offers a functionality for defining a structure of the messages generated by LLMs, AutoGen enables this functionality by propagating `response_format` passed to your agents to the underlying client.

For more info on structured output, please check [here](https://platform.openai.com/docs/guides/structured-outputs)


````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://ag2ai.github.io/ag2/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o", "gpt-4o-mini"],
    },
)

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Example: math reasoning

Using structured output, we can enforce chain-of-thought reasoning in the model to output an answer in a structured, step-by-step way

### Define the reasoning model

First we will define the math reasoning model. This model will indirectly force the LLM to solve the posed math problems iteratively trough math reasoning steps.

In [2]:
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

### Define chat actors

Now we can define the agents that will solve the posed math problem. 
We will keep this example simple; we will use a `UserProxyAgent` to input the math problem and an `AssistantAgent` to solve it.

The `AssistantAgent` will be constrained to solving the math problem step-by-step by using the `MathReasoning` response format we defined above.

In [5]:
llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,
    response_format=MathReasoning,
)

### Start the chat

Let's now start the chat and prompt the assistant to solve a simple equation. The assistant agent should return a response solving the equation using a step-by-step `MathReasoning` model.

In [6]:
user_proxy.initiate_chat(assistant, message="how can I solve 8x + 7 = -23", max_turns=1, summary_method="last_msg")

User_proxy (to Math_solver):

how can I solve 8x + 7 = -23

--------------------------------------------------------------------------------
[autogen.oai.client: 11-29 23:42:48] {921} INFO - Failed to cache response: Can't pickle <class 'openai.types.chat.parsed_chat_completion.ParsedChatCompletion[MathReasoning]'>: attribute lookup ParsedChatCompletion[MathReasoning] on openai.types.chat.parsed_chat_completion failed
Math_solver (to User_proxy):

{"steps":[{"explanation":"We need to isolate the variable x on one side of the equation. We start by getting rid of the constant on the left side of the equation. In this case, the constant is 7, which is added to 8x. We can subtract 7 from both sides of the equation to eliminate this constant.","output":"8x + 7 - 7 = -23 - 7\n8x = -30"},{"explanation":"Now that we have 8x = -30, we want to solve for x. To do this, we need to divide both sides of the equation by 8, which is the coefficient of x.","output":"8x / 8 = -30 / 8\nx = -30 / 8"},{"ex

ChatResult(chat_id=None, chat_history=[{'content': 'how can I solve 8x + 7 = -23', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '{"steps":[{"explanation":"We need to isolate the variable x on one side of the equation. We start by getting rid of the constant on the left side of the equation. In this case, the constant is 7, which is added to 8x. We can subtract 7 from both sides of the equation to eliminate this constant.","output":"8x + 7 - 7 = -23 - 7\\n8x = -30"},{"explanation":"Now that we have 8x = -30, we want to solve for x. To do this, we need to divide both sides of the equation by 8, which is the coefficient of x.","output":"8x / 8 = -30 / 8\\nx = -30 / 8"},{"explanation":"Next, we simplify the fraction -30 / 8. Both the numerator and the denominator can be divided by 2 to simplify the fraction.","output":"x = -30 / 8 = -15 / 4"},{"explanation":"The simplified fraction -15 / 4 can also be written as a decimal if needed.","output":"x = -15 / 4 = -3.75"}],"final_answe